In [ ]:
%matplotlib inline

import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from cselect import color as cs
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../samples')
from mvpose.data import epfl_campus
from time import time

root = Settings['data_root']
root = join(root, 'pak')
tmp = Settings['tmp']

import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.settings import get_settings
from paf_loader import Loader
from mvpose.evaluation import pcp
from mvpose.data import umpm

root = join(Settings['data_root'], 'pak')
user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']
tmp = Settings['tmp']
tmp = '/tmp'

params = get_settings()
loader = Loader(with_gpu=True)

X, Y, Calib = umpm.get(root, 'p2_free_1', user, pwd)

/home/tanke/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def generate_pcp_score(frame):
    alpha = 0.3
    Im = np.array([X[0][frame], X[1][frame], X[2][frame], X[3][frame]])
    heatmaps, pafs = loader.load_confidence_map_and_paf('umpm', 
                                                            Im, 
                                                            frame, 
                                                            dir=tmp)
    
    detections = pose.estimate(
        Calib, heatmaps, pafs, settings=params, debug=False)
    Humans = kth.transform3d_from_mscoco(detections)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []
    a = Y[frame,0:14,0:3]
    b = Y[frame,14:,0:3]
    Y_ = kth.transform3d_from_umpm([a, b])
    
    for gtid, gt in enumerate(Y_):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs, detections


Detections = []
PER_GTID = {}
n = 2437
#n = 20
for frame in range(0, n, 5):
    _start = time()
    L_Arms, U_Arms, L_Legs, U_Legs, GTIDs, detections =\
        generate_pcp_score(frame)
    Detections.append(detections)
    _end = time()
    print('frame ' + str(frame) + ', elapsed', _end - _start)
    
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['frame'].append(frame)

In [ ]:
total_avg = []
for key, values in PER_GTID.items():
    print('actor ', key)
    print('\tuarms:', np.mean(values['uarms']))
    print('\tlarms:', np.mean(values['larms']))
    print('\tulegs:', np.mean(values['ulegs']))
    print('\tllegs:', np.mean(values['llegs']))
    avg = np.mean([
        np.mean(values['uarms']),
        np.mean(values['larms']),
        np.mean(values['ulegs']),
        np.mean(values['llegs'])
    ])
    total_avg.append(avg)
    print('\tavg:  ', avg)
print('\navg*:  ', np.mean(total_avg))

## Like in the Master thesis
### alpha=0.5
* actor  0
	* uarms: 0.9764344262295082
	* larms: 0.9641393442622951
	* ulegs: 0.9897540983606558
	* llegs: 0.985655737704918
	* avg:   0.9789959016393442
* actor  1
	* uarms: 0.9682377049180327
	* larms: 0.9334016393442623
	* ulegs: 0.9877049180327869
	* llegs: 0.9805327868852459
	* avg:   0.967469262295082
* avg*:   0.9732325819672132

### alpha=0.2
* actor  0
	* uarms: 0.7510245901639344
	* larms: 0.6875
	* ulegs: 0.9743852459016393
	* llegs: 0.867827868852459
	* avg:   0.8201844262295082
* actor  1
	* uarms: 0.6997950819672131
	* larms: 0.7008196721311475
	* ulegs: 0.9323770491803278
	* llegs: 0.9293032786885246
	* avg:   0.8155737704918034
* avg*:   0.8178790983606558

## Like in Master thesis without Unaries
### alpha=0.5
* actor  0
	* uarms: 0.9774590163934426
	* larms: 0.9651639344262295
	* ulegs: 0.9897540983606558
	* llegs: 0.9866803278688525
	* avg:   0.979764344262295
* actor  1
	* uarms: 0.9692622950819673
	* larms: 0.9323770491803278
	* ulegs: 0.9877049180327869
	* llegs: 0.9805327868852459
	* avg:   0.967469262295082
* avg*:   0.9736168032786885

### alpha=0.2
* actor  0
	* uarms: 0.7510245901639344
	* larms: 0.6875
	* ulegs: 0.9733606557377049
	* llegs: 0.8668032786885246
	* avg:   0.819672131147541
* actor  1
	* uarms: 0.6987704918032787
	* larms: 0.6997950819672131
	* ulegs: 0.9313524590163934
	* llegs: 0.9282786885245902
	* avg:   0.8145491803278688
* avg*:   0.8171106557377049